In [17]:
import pandas as pd
from datetime import datetime, date
from dateutil.parser import parse

# !pip install --upgrade xlrd
# !pip install pyarrow
# !pip install openpyxl

In [18]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)

In [19]:
# # https://fred.stlouisfed.org/series/FEDFUNDS

# today = date.today().strftime("%Y-%m-%d")
# df = pd.read_excel(f'https://fred.stlouisfed.org/graph/fredgraph.xls?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=FEDFUNDS&scale=left&cosd=1954-07-01&coed=2022-06-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date={today}&revision_date={today}&nd=2017-01-01', header=10, names=['date','FEDrate'])
# df = df[df['date'] >= datetime(year=2017, month=1, day=1)]
# df = df.reset_index(drop=True)
# df['year'] = df['date'].apply(lambda x: x.year)
# df['month'] = df['date'].apply(lambda x: x.month)
# df['day'] = df['date'].apply(lambda x: x.day)
# df.to_feather('Data/FEDFUNDS.ftr')
# df.head()

In [20]:
# # https://fred.stlouisfed.org/series/EFFRVOL

# df = pd.read_excel('Data/EFFRVOL.xls', header=10, names=['date', 'EFFRVol'])
# df['year'] = df['date'].apply(lambda x: x.year)
# df['month'] = df['date'].apply(lambda x: x.month)
# df['day'] = df['date'].apply(lambda x: x.day)
# df.to_feather('Data/EFFRVOL.ftr')
# df.head()

In [21]:
# df = pd.read_excel('Data/SP500.xlsx', header=5)
# df = df.iloc[:,0:2].copy()
# df.columns = ['date','last_price']
# df['year'] = df['date'].apply(lambda x: x.year)
# df['month'] = df['date'].apply(lambda x: x.month)
# df['day'] = df['date'].apply(lambda x: x.day)
# df.to_feather('Data/SP500.ftr')
# df.head()

In [22]:
# # https://download.bls.gov/pub/time.series/cu/cu.txt

# CPI_types = ['11.USFoodBeverage', '12.USHousing', '13.USApparel', '14.USTransportation', '15.USMedical', '16.USRecreation', '17.USEducationAndCommunication', '18.USOtherGoodsAndServices', '20.USCommoditiesServicesSpecial']
# CPI_dict = {}

# for name in CPI_types:
#     df = pd.read_csv(f'https://download.bls.gov/pub/time.series/cu/cu.data.{name}', delimiter='\t')
#     df = df.iloc[:,:4]
#     df.columns = ['series_id','year','month','CPI']
#     df = df[df['year'] >= 2017]
#     df['month'] = df['month'].apply(lambda x: int(x[1:]))
#     df = df[df['month'] != 13]
#     df = df.reset_index(drop=True)

#     new_CPI = []
#     for y in df['year'].unique():
#         for m in df['month'].unique():
#             year = df[df['year']==y]
#             month = year[year['month']==m]
#             thingy = (y, m, month['CPI'].mean())
#             new_CPI.append(thingy)
#     new_name = name.split('.')[1]
#     CPI_dict[f'{name}'] = pd.DataFrame(new_CPI, columns=['year', 'month', f'CPI_{new_name}'])


# df = CPI_dict[list(CPI_dict.keys())[0]][['year', 'month']]
# for key in list(CPI_dict.keys()):
#     df[key] = CPI_dict[key].iloc[:,2]

# df['day'] = 1
# date_creation = df[['year', 'month', 'day']].astype(str).copy()
# date_creation['date'] = date_creation['year']+'-'+date_creation['month']+'-'+date_creation['day']
# df['date'] = date_creation['date'].apply(parse)
# df = df.reindex(columns=['date', 'year','month','day','11.USFoodBeverage', '12.USHousing', '13.USApparel', '14.USTransportation', '15.USMedical', '16.USRecreation', '17.USEducationAndCommunication', '18.USOtherGoodsAndServices', '20.USCommoditiesServicesSpecial'])

# df.to_feather('Data/CPI.ftr')
# df.head()

In [23]:
a = pd.read_feather('Data/FEDFUNDS.ftr')
b = pd.read_feather('Data/EFFRVOL.ftr')
c = pd.read_feather('Data/SP500.ftr')
d = pd.read_feather('Data/CPI.ftr')

# Merging datasets, filling in values of missing dates using the previous value.
joined = pd.merge_asof(c, a, on='date', by=['year', 'month', 'day'])
joined = pd.merge_asof(joined, b, on='date', by=['year', 'month', 'day'])
joined = pd.merge_asof(joined, d, on='date', by=['year', 'month', 'day'])
joined = joined.reset_index(drop=True)

# There were some places where EFFRVol = 0, so I filled them with the mean of preceding and following values
for i in range(len(joined)):
    if joined['EFFRVol'][i] == 0:
        joined['EFFRVol'][i] = (joined['EFFRVol'][i-1] + joined['EFFRVol'][i+1]) / 2

joined = joined.fillna(method='ffill')
joined = joined.fillna(method='bfill')

# Creating a column to signal when (and how) the Fed changes rate
FEDrate_delta = []
for i in range(len(joined) - 1):
    FEDrate_delta.append(joined['FEDrate'][i+1] - joined['FEDrate'][i])
FEDrate_delta.append(0)
joined['FEDrate_delta'] = FEDrate_delta

joined.to_feather('Data/cleaned_dataset.ftr')
joined.head()

C:\Users\ethan\AppData\Local\Temp\ipykernel_18876\3757469383.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined['EFFRVol'][i] = (joined['EFFRVol'][i-1] + joined['EFFRVol'][i+1]) / 2


,date,last_price,year,month,day,FEDrate,EFFRVol,11.USFoodBeverage,12.USHousing,13.USApparel,14.USTransportation,15.USMedical,16.USRecreation,17.USEducationAndCommunication,18.USOtherGoodsAndServices,20.USCommoditiesServicesSpecial,FEDrate_delta
0,2017-01-02,2238.83,2017,1,2,0.66,80.0,213.09825,186.539952,121.932528,181.273422,342.144315,130.226242,216.391311,255.196766,211.14058,0.0
1,2017-01-03,2257.83,2017,1,3,0.66,80.0,213.09825,186.539952,121.932528,181.273422,342.144315,130.226242,216.391311,255.196766,211.14058,0.0
2,2017-01-04,2270.75,2017,1,4,0.66,79.0,213.09825,186.539952,121.932528,181.273422,342.144315,130.226242,216.391311,255.196766,211.14058,0.0
3,2017-01-05,2269.00,2017,1,5,0.66,77.0,213.09825,186.539952,121.932528,181.273422,342.144315,130.226242,216.391311,255.196766,211.14058,0.0
4,2017-01-06,2276.98,2017,1,6,0.66,79.0,213.09825,186.539952,121.932528,181.273422,342.144315,130.226242,216.391311,255.196766,211.14058,0.0
